# LangChain Basic

ollama을 제외한 상용 AI들은 비용이 든다.  

따라서, 비용이 없이 LLM을 사용하기 위해서는 open source로 제공되는 ollama를 활용하여 진행할 수 있지만, 성능은 상용 LLM보단 조금 떨어진다고 한다.



# 1. Install Ollama[(🔗Link)[https://ollama.com/]]

(🔗해당 링크)[https://ollama.com/]를 통해서 ollama를 설치한다.

ollama에서 제공하는 수많은 LLM model이 있는데 난 강의에서 사용하는 deepseek-r1아 아닌 meta에서 2024년 12월에 발표한 `llama3.3`을 사용하려고 했지만 용량이 크고 text를 위주로 할 것이기 때문에 `3.1`를 사용한다.


ollama -> models -> llama3.1을 찾아서 정보를 확인후 아래 command를 통해서 local에 다운받는다.
```shell
$) ollama pull llama3.1
```

In [ ]:
# %pip install -Uq langchain-ollama

Note: you may need to restart the kernel to use updated packages.


In [1]:
from langchain_ollama import ChatOllama

llm = ChatOllama(model="llama3.1")
llm.invoke("대한민국의 수도 알려줘")

AIMessage(content='서울입니다.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-07-20T06:01:49.508797Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1170241542, 'load_duration': 767241209, 'prompt_eval_count': 17, 'prompt_eval_duration': 340222541, 'eval_count': 4, 'eval_duration': 61469375, 'model_name': 'llama3.1'}, id='run--0bcc0468-0a95-47cf-8020-cbb1dd67d991-0', usage_metadata={'input_tokens': 17, 'output_tokens': 4, 'total_tokens': 21})

# 2. prompt

위에서 llm.invoke에 들어가는 인자는 `prompt`로서 llm에 질의하기 위한 interface라고 생각하면 된다.   
prompt로 들어갈 수 있는 type은 아래 `3가지`이다.

1. Prompt Value => `langchain의 prompt template`을 사용하여 만들 수 있음
2. string
3. list of BaseMessage  

## 2.1 Prompt Value를 통해 질의

In [9]:
from langchain_core.prompts import PromptTemplate

# * "country"라는 변수를 받아서 template에 있는 country로 넣어준다.
# * 아래 template.invoke를 통해서 들어간다.
prompt_template = PromptTemplate(
  template="What is the capital of {country}?",
  input_variables=["country"]
)

prompt = prompt_template.invoke({"country": "Korea"})
llm = ChatOllama(model="llama3.1")
llm.invoke(input=prompt)

AIMessage(content='There are actually two countries called "Korea": North Korea (Democratic People\'s Republic of Korea) and South Korea (Republic of Korea).\n\nThe capital of:\n\n* **North Korea** is Pyongyang.\n* **South Korea** is Seoul.\n\nSo, which one were you thinking of?', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-07-20T05:56:13.931602Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1365230625, 'load_duration': 80469584, 'prompt_eval_count': 17, 'prompt_eval_duration': 156762875, 'eval_count': 58, 'eval_duration': 1127568667, 'model_name': 'llama3.1'}, id='run--9e2c5e7c-71a7-4f88-863a-f8b1c263fd84-0', usage_metadata={'input_tokens': 17, 'output_tokens': 58, 'total_tokens': 75})

## 2.2 List of BaseMessage를 통한 질의

BaseMessage를 상속하는 클래스들이 있는데 `4 가지`가 존재하면 아래와 같다.

1. HumanMessage => 사용자의 질의
2. AIMessage => LLM의 답변
3. SystemMessage => prompt engineering에서 페르소나 때 사용
4. ToolMessage => AI agent에서 사용

아래에서 `prompt engineering` 기법 중 few-shots, zero-shot, one-shot과 같은 기법이 나오는데 여기서 `shot = 예시 또는 예제; 즉, 질문-답변 한 쌍`이라고 생각하면 된다.  

아래 [🔗논문](https://arxiv.org/abs/2005.14165)에서 발췌한 이미지이다.  

![what is shot](./few-shots.png)

위의 이미지에서 예시를 많이주면 `답변의 정확도가 높아짐`을 논문에서 발표한다.








In [ ]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage

# * BaseMessage를 상속하는 HumanMessage를 사용하여 질의
base_msg_result = llm.invoke([HumanMessage(content="대한민국의 수도는 어디인가요?")])

# * BaseMessage를 상속하는 것을 넣어줄 수 있다.
# ! 아래에서 HumanMessage가 마지막에 와야 질의에 대한 응답을 얻을 수 있다.
# ! AIMessage가 마지막에 오면 결국 답변이기 때문에 답변을 받을 수 없다.
# ? 아래와 같이 SystemMessage, HumanMessage, AIMessage를 넣는 이유는 AI에게 질의에 대한 history가 있는 것처럼 속이기 위함이다.
# ? 이를 통해서 답변을 우리가 원하는대로 유도할 수 있다.
# ? 아래는 질문-답변이 한 쌍이 존재하므로 "one-shot"이다.
prompt_msg_list = [
  SystemMessage(content="당신은 세계에서 최고 유명한 지리학자이자 박사입니다."),
  HumanMessage(content="대한민국의 수도는 어디인가요?"),
  AIMessage(content="대한민국의 수도는 서울입니다."),
  HumanMessage(content="대한민국의 수도는 어디인가요?"),  
]

llm.invoke(prompt_msg_list)

AIMessage(content='네, 한국의 수도는 서울특별시입니다.', additional_kwargs={}, response_metadata={'model': 'llama3.1', 'created_at': '2025-07-20T06:19:35.103928Z', 'done': True, 'done_reason': 'stop', 'total_duration': 406587375, 'load_duration': 33495375, 'prompt_eval_count': 69, 'prompt_eval_duration': 193075209, 'eval_count': 10, 'eval_duration': 178859416, 'model_name': 'llama3.1'}, id='run--8d686e25-d5a5-4d92-98ba-6747ede5b642-0', usage_metadata={'input_tokens': 69, 'output_tokens': 10, 'total_tokens': 79})

### 메세지 리스트를 사용하기 위해선 `ChatPromptTemplate`를 사용

위의 코드보다 추후 `LCEL` 파이프라인 기법에 사용할 수 없으므로  
_확장성 측면에서_ 아래와 같이 `ChatPromptTemplate`를 통한 message_list를 구성하는 것이 좋다.

In [11]:
from langchain_core.prompts import ChatPromptTemplate

# ! 위에서처럼 리스트의 요소가 Human, AI, System이 아닌 "tuple"로 넣어주어야 country가 사용자의 값으로 대체가 된다.
base_msg_list = [
  ("assistant", "당신은 세계에서 최고 유명한 지리학자이자 박사입니다."),
  ("human", "대한민국의 수도는 어디인가요?"),
  ("ai", "대한민국의 수도는 서울입니다."),
  ("human", "{country} 수도는 어디인가요?"),
]
chat_prompt_template = ChatPromptTemplate.from_messages(base_msg_list)

chat_prompt_template_result = chat_prompt_template.invoke({"country": "대한민국"})
print("chat_prompt_template_result =>", chat_prompt_template_result)
result = llm.invoke(chat_prompt_template_result)
print("result =>", result)


chat_prompt_template_result => messages=[AIMessage(content='당신은 세계에서 최고 유명한 지리학자이자 박사입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='대한민국의 수도는 어디인가요?', additional_kwargs={}, response_metadata={}), AIMessage(content='대한민국의 수도는 서울입니다.', additional_kwargs={}, response_metadata={}), HumanMessage(content='대한민국 수도는 어디인가요?', additional_kwargs={}, response_metadata={})]
result => content='서울입니다.' additional_kwargs={} response_metadata={'model': 'llama3.1', 'created_at': '2025-07-20T06:26:18.488727Z', 'done': True, 'done_reason': 'stop', 'total_duration': 301085666, 'load_duration': 79030541, 'prompt_eval_count': 68, 'prompt_eval_duration': 158884250, 'eval_count': 4, 'eval_duration': 61395750, 'model_name': 'llama3.1'} id='run--0304c60f-41c8-47b5-a9cc-2ba2f5cc0aa8-0' usage_metadata={'input_tokens': 68, 'output_tokens': 4, 'total_tokens': 72}


# pydantic과 연계
=> `section4(6:30)`에서 json output parser를 사용해도 제대로 된 json이 나오지 않아 `pydantic`과 연계하여 `output based schema with pydantic`을 할 수 있다.

```python
from pydantic import BaseModel, Field

class CountryDetail(BaseModel):
  capital: str = Field(description="The capital of the country")
  population: int = Field(description="The population of the country")
  language: str = Field(description="The language of the country")
  currency: str = Field(description="The currency of the country")

county_detail_prompt = PromptTemplate(
  template="""
  Give following information about {country}:
  - Capital
  - Population
  - Language
  - Currency

  return it in JSON format. and return the JSON dictionary only.
  """,
  input_variables=["country"]
)

# ! json ouput parser 대신 pydantic과 with_structured_output을 연계하여작성하자
structured_llm = llm.with_structured_output(CountryDetail)
json_ai_message = structured_llm.invoke(county_detail_prompt.invoke({"country": "France"}))

# 이와 같이 하면 아래와 같이 각 property에 쉽게 접근할 수 있다.
json_ai_message.capital
json_ai_message.population
json_ai_message.language
json_ai_message.currency
```




# chain끼리 연결

최종 chain은 사용자로부터 받은 국가 이름을 통해 국가의 정형화된 데이터를 추출하고 이를 기반으로 나라 국기의 색깔을 추론하는 chain이다.

> final_chain = country_llm_chain(input_variables=["country"]) => country_flag_colors_chain(input_variables=["information"])

- country_llm_chain은 {"country": "France"}와 같이 넣어주어야 함
- country_flag_colors_chain은 {"information": f"{information}"}와 같이 넣어주어야 함

`RunnablePassthrough`를 통해서 key가 없이 바로 값을 넣어주고 `RunnableLambda`를 통해서 중간 결과를 출력하는 custom function pipeline을 만들 수 있다.


## country_llm_chain 구현

`pydantic`과 `with_structured_output`을 통해서 국가의 정형화된 데이터를 추출한다.
- 수도
- 인구
- 언어
- 통화

In [19]:
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field

load_dotenv()

template = """
  Give following information about {country}:
  - Capital
  - Population
  - Language
  - Currency

  return it in JSON format. and return the JSON dictionary only.
  """

class CountryDetail(BaseModel):
  capital: str = Field(description="The capital of the country")
  population: int = Field(description="The population of the country")
  language: str = Field(description="The language of the country")
  currency: str = Field(description="The currency of the country")

llm = ChatOpenAI(model="gpt-4o-mini").with_structured_output(CountryDetail)
prompt = PromptTemplate(
  template=template,
  input_variables=["country"]  
)

country_llm_chain = prompt | llm

# ! 중간 test
# answer = country_llm_chain.invoke({"country": "France"})


## country_flag_colors_chain 구현


In [ ]:
from langchain_core.output_parsers import StrOutputParser


capital_template = PromptTemplate(
  template="""
  guess country's flag colors.
  just return the colors.

  # information
  {information}
  """,
  input_variables=["information"]
)

output_parser = StrOutputParser()
country_flag_colors_llm = ChatOpenAI(model="gpt-4o-mini")
country_flag_colors_chain = capital_template | country_flag_colors_llm | output_parser


## RunnableLambda를 이용한 중간 값 출력

In [23]:
from langchain_core.runnables import RunnablePassthrough, RunnableLambda

def print_intermediate_output(answer):
  print("중간 값 출력 =>", answer)

  return answer


final_chain = {"information": country_llm_chain} | RunnableLambda(print_intermediate_output) |  country_flag_colors_chain 
final_chain

{
  information: PromptTemplate(input_variables=['country'], input_types={}, partial_variables={}, template='\n  Give following information about {country}:\n  - Capital\n  - Population\n  - Language\n  - Currency\n\n  return it in JSON format. and return the JSON dictionary only.\n  ')
               | RunnableBinding(bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x112117950>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x112116b10>, root_client=<openai.OpenAI object at 0x112117ce0>, root_async_client=<openai.AsyncOpenAI object at 0x112115a70>, model_name='gpt-4o-mini', model_kwargs={}, openai_api_key=SecretStr('**********')), kwargs={'response_format': <class '__main__.CountryDetail'>, 'ls_structured_output_format': {'kwargs': {'method': 'json_schema', 'strict': None}, 'schema': {'type': 'function', 'function': {'name': 'CountryDetail', 'description': '', 'parameters': {'properties': {'capital':

## RunnablePassthrough를 이용한 편한 입력 구현

그냥 단일값을 넣어줄 때는 key값을 생략가능하지만 만약 country_llm_chain이 `두 개 이상 input_variables`를 받는다면 key, value를 모두 작성해야 한다.

```python
final_chain = {
  "country": RunnablePassthrough(),
  "continet": RunnablePassthrough()
} | country_llm_chain | RunnableLambda(print_intermediate_output) | country_flag_colors_chain 

answer = final_chain.invoke({
  "country": "France",
  "continent": "Europe"
})
```

In [24]:
from langchain_core.runnables import RunnablePassthrough

final_chain = {"country": RunnablePassthrough()} | country_llm_chain | RunnableLambda(print_intermediate_output) | country_flag_colors_chain 

In [ ]:

# RunnablePassthrough 사용 전
# e.g. answer = final_chain.invoke({"country": "France"})
answer = final_chain.invoke("France")
print(answer)

중간 값 출력 => capital='Paris' population=65273511 language='French' currency='Euro'
Blue, White, Red


# prompt 꿀팁

사람과 같이 llm은 무엇을 해야하게끔 설계가 되어있는데 사용자들은 `하지말라는 것`과 `해야 하는 것`을 명령한다.  
따라서, `하지 말아야하는 것(safety)`은 별도의 chain을 구성하고 `pydantic`과 같이 `binary result`가 나오게끔 구현을 하고 나서 `통과시에만` 다음 chain으로 넘어가게 한다.

이렇게 하면 safety chain은 간단한 결과를 출력하므로 적은 비용의 모델을 사용, 이후 실제 logic이 돌아가는 chain은 성능이 중요하기 때문에 고 비용의 모델과 같이 사용할 수 있다.

```plaintext
safety_chain(low cost model) ---- no -> ❌
             ㄴ--- yes -> next_chain(high cost model) -> ✅
```

_**<span style="color: orange;">또한 prompt는 하나의 task를 처리하도록 하고 이를 조합하여 chain을 구성하는 것이 하나의 Prompt에 여러 질문을 넣어 처리하는 것 보다 훨씬 성능이 좋다고 한다.</span>**_







# Knowledge cutoff

llm은 과거의 데이터를 학습한 언어 모델이다.
- gpt-4o가 학습한 데이터의 마지막 시점을 `knowledge cutoff`라고 한다.
- 즉, gpt-4o는 2023년 10월 이전의 데이터만 학습한 모델이다.

하지만, `legal: 법`과 같은 도메인에 대한 언어 모델을 구축하는 것은 새로운 법 개정안이 나올 때마다 학습을 시켜주어야 한다. 

따라서, 새로운 학습 데이터를 사람이 찾아서 주어야 하는데 이 때 `RAG`를 활용하면 해결할 수 있다.


> 시대 흐름:  `perception AI` -> `Generative AI` -> `Agent AI` -> `Robot`


